1) Для головоломок типа "Кубик" мы знаем правила 🎲 __mi.mi.mi = -mi__ и __mi.mi.mi.mi = пусто__. Давайте расширим их для всех трёх видов головоломок, создав правила изменения знака в форме __mi\*\*(n) = -mi\*\*(n)__:

   - Для "Кубика", __n=2__, или __mi.mi = -mi.-mi__. Используя правило отмены пар (https://www.kaggle.com/code/cl12102783/cancel-pairs-for-all-puzzles), получаем:
     - mi.mi.mi = -mi.-mi.mi = -mi
     - mi.mi.mi.mi = -mi.-mi.mi.mi = пусто
     - Также существует коммутативное правило для "Кубика": mi.m?.mi.m?.mi = неупорядоченный(-mi.m?.m?)
   
   - Для "Венка", n зависит от его размера:
     - Если размер венка чётный, __n = размер/2__. Например, wreath_6/6 имеет n=3
     - Если размер венка нечётный, __n = размер__. Например, wreath_7/7 имеет n=7
     - "Венок" некоммутативен, поэтому замена производится без изменения местоположения, аналогично https://www.kaggle.com/code/cl12102783/cancel-pairs-for-all-puzzles
   
   - Для "Глобуса":
     - fi некоммутативно с fi=-fi (https://www.kaggle.com/code/cl12102783/cancel-pairs-for-all-puzzles), или n=1
     - ri коммутативно, n равно конечному числу типа. Например, globe_1/8 имеет n=8

2) Кроме того, для применения правила изменения знаков, когда количество mi превышает n, происходит уменьшение. Правило уменьшения: __n_left = n-(count - n)__:
   - Если n_left>0, то будет __-mi\*\*(n_left)__. Например, для "Кубика" mi.mi.mi имеет n=2, так что 2-(2-1)=1, становится -mi 🔄
   - Если n_left<0, то будет __mi\*\*(n_left)__. Например, для globe_1/8, ri\*\*(18) имеет n=8, так что 8-(18-8)=-2, становится ri\*\*(2) 🔀
   - Если n_left=0, то все mi отменяются в одной группе ходов. Например, для "Кубика" mi.mi.mi.mi имеет n=2, так что 2-(4-2)=0, отменяются 🚫


In [69]:
# Импорт необходимых библиотек
import pandas as pd
from collections import deque
import tqdm
import glob
import numpy as np
from collections import Counter

# Задание пути к файлам данных
path = '/kaggle/input/santa-2023/'

# Загрузка данных из CSV файлов в pandas DataFrame
df_puzzles = pd.read_csv(path + 'puzzles.csv')
df_puzzle_info = pd.read_csv(path + 'puzzle_info.csv')

# Выбор всех файлов с высокими результатами
files = [i for i in glob.glob('/kaggle/input/*/*.csv') if 'submission' in i and '/santa-2023/' not in i]


In [70]:
def optimized_cancel_group(moves, group):
    def get_grp(elem):
        return elem[1] if elem.startswith('-') else elem[0]

    def optimize_moves(elem, group):
        grp = group.split('_')[0]
        grp_size = {'cube': 2, 'globe': int(group.split('/')[-1])}.get(grp, 2)
        move_count = Counter(elem)

        optimized_moves = []
        for move, count in move_count.items():
            if count > grp_size:
                excess = count - grp_size
                if grp != 'globe' or move[-1].replace('-', '')[0] != 'f':
                    move = '-' + move.replace('-', '') if count % 2 != 0 else move
                    optimized_moves.append(move * (excess % grp_size))
            else:
                optimized_moves.append(move * count)

        return optimized_moves

    # Обработка ходов
    moves += '.'
    grouped_moves = []
    current_group = []

    for char in moves:
        if char != '.':
            current_group.append(char)
        else:
            if current_group:
                optimized_group = optimize_moves(''.join(current_group), group)
                grouped_moves.extend(optimized_group)
                current_group = []

    return '.'.join(grouped_moves)

def multiple_try_optimized(elem, group):
    prev_len = len(elem.split('.'))
    optimized_move = elem

    while True:
        new_move = optimized_cancel_group(optimized_move, group)
        new_len = len(new_move.split('.'))

        if new_len < prev_len:
            optimized_move = new_move
            prev_len = new_len
        else:
            break

    return optimized_move

# # Применение функции
# result = {}
# for file in tqdm.tqdm(files):
#     with open(file, 'r') as f:
#         next(f, None)
#         for row in f:
#             id_, move = row.strip().split(',')
#             id_ = int(id_)
#             group = df_puzzles.at[id_, 'puzzle_type']
#             move = multiple_try_optimized(move, group)
#             result[id_] = move

# df_sub = pd.DataFrame({'id': result.keys(), 'moves': result.values()})
# df_sub.to_csv('submission.csv', index=False)


Этот код выполняет следующие действия:

Создает словарь result для хранения результатов.
Итерирует по файлам, используя tqdm для отслеживания прогресса.
Открывает каждый файл и обрабатывает строки, игнорируя заголовок.
Извлекает идентификатор (id_) и ход (move) из каждой строки.
Определяет тип головоломки (group) для данного идентификатора из DataFrame df_puzzles.
Применяет функцию multiple_try к ходу.
Обновляет результат в словаре result, учитывая наименьшую длину хода.


In [72]:
from functools import lru_cache
# # Создание словаря для хранения результатов
result = {}

@lru_cache(maxsize=None)
def multiple_try_cached(elem, group):
    return multiple_try(elem, group)

# Использование кэшированной версии функции в основном коде
for file in tqdm.tqdm(files):
    # Открытие файла для чтения с использованием контекстного менеджера
    with open(file, 'r') as f:
        # Игнорирование заголовка сразу после открытия файла
        next(f, None)
        for row in f:
            # Извлечение данных из строки
            id_, move = row.strip().split(',')
            id_ = int(id_)
            # Определение типа головоломки для данного идентификатора без использования DataFrame
            group = df_puzzles.at[id_, 'puzzle_type']
            # Применение функции multiple_try к ходу
            move = multiple_try_cached(move, group)
            # Остальной код остается неизменным
            
            
                        # Обновление результата в словаре
            if id_ not in result:
                result[id_] = move
            else:
                # Сравнение ходов и выбор наименьшего по длине
                if len(move.split('.')) < len(result[id_].split('.')):
                    result[id_] = move


100%|██████████| 3/3 [00:08<00:00,  2.70s/it]


In [73]:
df_sub = pd.DataFrame()
df_sub['id'] = result.keys()
df_sub['moves'] = result.values()
df_sub.to_csv('submission.csv', index=False)

In [74]:
df_sub.moves.str.split('.').apply(lambda x: len(x)).sum()

827041